In [1]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/25740/head accelerate

  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/25740/head) to /tmp/pip-req-build-k85m3f0g
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-k85m3f0g
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/25740/head
  Running command git checkout -q c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Resolved https://github.com/huggingface/transformers.git to commit c6c6daa3f07e753cff91a08c4294df4a6ea6227b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7587021 sha256=4eb227a02e29ddf9f46df579a234cbdd5545341fb61e5868cb5601698b1

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = "codellama/CodeLlama-7b-Instruct-hf"
model = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [4]:
!pip install -q datasets # install HF datasets library
from datasets import load_dataset
import tensorflow as tf


url = "https://raw.githubusercontent.com/google-research/google-research/master/mbpp/mbpp.jsonl" # download mbpp dataset
dataset_path = tf.keras.utils.get_file("mbpp.jsonl", origin = url, cache_dir="../working/", cache_subdir="../working/")

dataset = load_dataset('json', data_files=dataset_path)['train']
tests_count = 100
start = 11 # в репозитории mbpp сказано, что для тестирования предназачаются задачи 11-510
end = start + tests_count
dataset = dataset[start : end]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
563743/563743 [==============================] - 0s 0us/step


Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
prompt_list = []
for i in range(tests_count):
  prompt_list.append(dataset['text'][i] + ' The code should also pass these tests: ' +  dataset['test_list'][i][0] + ', ' + dataset['test_list'][i][1])

In [14]:
# Подаю в модель запрос следующего вида
prompt_list[0]

'Write a function to sort a given matrix in ascending order according to the sum of its rows. The code should also pass these tests: assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]], assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]'

In [ ]:
model.eval()
responses = []

for prompt in prompt_list:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=400)
        response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    responses.append(response)

In [36]:
def extract_function(text):
    function_lines = []
    inside_function = False

    for line in text.split('\n'):
        if line.strip().startswith("def "):  # начало функции
            function_lines.append(line)
            inside_function = True
        elif inside_function:
            function_lines.append(line)
            if line.strip() == "":  # конец функции
                inside_function = False
                function_code = '\n'.join(function_lines)
                return function_code

In [41]:
score = 0
for i in range(tests_count):
  code = extract_function(responses[i])
  if code:
    code += '\n' + dataset['test_list'][i][2]
    code = code.strip()
    try:
      exec(code)
    except (AssertionError, TypeError):
      #print(f"Код {i} не прошел тест")
      pass
    else:
      #print(f"Код {i} прошел тест")
      score += 1
print(score / tests_count)

0.49
